# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')

In [4]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [6]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [9]:
def label_fix(label):
    if label == '<=50K':
        return 0
    else:
        return 1

In [10]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
data = census.drop(labels='income_bracket',axis=1)

In [13]:
labels = census['income_bracket']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [35]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [36]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [49]:
gender = tf.feature_column.categorical_column_with_vocabulary_list(key="gender",vocabulary_list=["Female","Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket(key="occupation",hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(key = "marital_status",hash_bucket_size=1000)
realtionship = tf.feature_column.categorical_column_with_hash_bucket(key="relationship",hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket(key="education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket(key= "workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket(key="native_country",hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [50]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [39]:
# For DNNClassifier we need embedded columns

###embedded_col_group = ['gender','occupation','martial_status','relationship','education','workclass','native_country']

In [40]:
#embedded_col = tf.feature_column.embedding_column(categorical_column='embedded_col_group',dimension=10)

In [51]:
feat_col = [gender,occupation,marital_status,realtionship,education,workclass,native_country,age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [52]:
input_func = tf.estimator.inputs.pandas_input_fn(x_train,y_train,batch_size=128,shuffle=True,num_epochs=1)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [53]:
model = tf.estimator.LinearClassifier(n_classes=2,feature_columns=feat_col)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/01/g_8xj46j25v9kkjttk5p2j5h0000gn/T/tmpxs76_umj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a28c5ed68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [54]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/01/g_8xj46j25v9kkjttk5p2j5h0000gn/T/tmpxs76_umj/model.ckpt.
INFO:tensorflow:loss = 88.722855, step = 1
INFO:tensorflow:global_step/sec: 161.02
INFO:tensorflow:loss = 0.00024727182, step = 101 (0.623 sec)
INFO:tensorflow:global_step/sec: 333.908
INFO:tensorflow:loss = 0.0002637853, step = 201 (0.298 sec)
INFO:tensorflow:Saving checkpoints for 204 into /var/folders/01/g_8xj46j25v9kkjttk5p2j5h0000gn/T/tmpxs76_umj/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00013098403.


### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [57]:
input_eval_func =tf.estimator.inputs.pandas_input_fn(x_test,
                                                    batch_size=128,
                                                     num_epochs=1,
                                                     shuffle=False)
                                                 
                                                

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [60]:
prediction=list(model.predict(input_fn=input_eval_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/01/g_8xj46j25v9kkjttk5p2j5h0000gn/T/tmpxs76_umj/model.ckpt-204
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [62]:
prediction[0]

{'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logistic': array([0.9999889], dtype=float32),
 'logits': array([11.407445], dtype=float32),
 'probabilities': array([1.1112322e-05, 9.9998891e-01], dtype=float32)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [63]:
final_preds=[]
for pred in prediction:
    final_preds.append(pred['class_ids'][0])

In [64]:
final_preds[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [65]:
from sklearn.metrics import classification_report

In [66]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00      6513

avg / total       1.00      1.00      1.00      6513



# Great Job!